In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from tqdm import trange
import pandas as pd
import time as tm
import requests
import os

# Selenium

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')

s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options = options,service = s)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/home/cosbi/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache


In [3]:
driver.get("https://mops.twse.com.tw/mops/web/t100sb02_1")

In [4]:
select_element = driver.find_element(by = By.NAME, value = "month")
select_object = Select(select_element)
select_object.select_by_index(6)

In [5]:
driver.find_elements(by = By.XPATH, value = "//input[@type='button']")[1].click()

In [7]:
driver.find_elements(by = By.TAG_NAME, value = "button")[0].click()

# Bs4

In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [7]:
Id, name, date, time, place, message, ch_file, en_file, more_information, video_address, attention = [[] for x in range(11)]

In [8]:
result_even = soup.find_all("tr", class_ = "even")
result_odd = soup.find_all("tr", class_ = "odd")
result_total = result_even + result_odd

for i in trange(len(result_total)):
    data_td = result_total[i].find_all("td")

    for ptr in range(len(data_td)):
        if ptr == 0:
            Id.append(data_td[ptr].getText())
        elif ptr == 1:
            name.append(data_td[ptr].getText())
        elif ptr == 2:
            temp = data_td[ptr].getText().replace("/", "-")
            newYear = str(int(temp.split("-")[0]) + 1911)
            temp = temp.replace(temp.split("-")[0], "")
            temp = newYear + temp
            date.append(temp)
        elif ptr == 3:
            time.append(data_td[ptr].getText())
        elif ptr == 4:
            temp = data_td[ptr].getText()
            place.append(temp)
        elif ptr == 5:
            temp = data_td[ptr].getText()
            message.append(temp)
        elif ptr == 6:
            temp = data_td[ptr].getText()

            if ".pdf" in temp:
                download_payload = {
                    "step": "9",
                    "filePath": "/home/html/nas/STR/",
                    "fileName": str(temp),
                    "functionName": "t100sb02_1"
                }

                while(True):
                    download_response = requests.post("https://mops.twse.com.tw/server-java/FileDownLoad"
                                         , data = download_payload)

                    data = download_response.content
                    tm.sleep(1)

                    if(len(data) != 496):
                        break
                
                if not os.path.isdir("/home/cosbi/桌面/financialData/twseData/data/ch/" + Id[-1]):
                    os.mkdir("/home/cosbi/桌面/financialData/twseData/data/ch/" + Id[-1])
                
                with open("/home/cosbi/桌面/financialData/twseData/data/ch/" + Id[-1] + "/" + temp, 'wb') as s:
                    s.write(data)

                ch_file.append(temp)
            else:
                ch_file.append("null")
        elif ptr == 7:
            temp = data_td[ptr].getText()

            if ".pdf" in temp:
                download_payload = {
                    "step": "9",
                    "filePath": "/home/html/nas/STR/",
                    "fileName": str(temp),
                    "functionName": "t100sb02_1"
                }

                while(True):
                    download_response = requests.post("https://mops.twse.com.tw/server-java/FileDownLoad"
                                            , data = download_payload)
                    
                    data = download_response.content
                    tm.sleep(1)

                    if(len(data) != 496):
                        break
                
                if not os.path.isdir("/home/cosbi/桌面/financialData/twseData/data/en/" + Id[-1]):
                    os.mkdir("/home/cosbi/桌面/financialData/twseData/data/en/" + Id[-1])
                
                with open("/home/cosbi/桌面/financialData/twseData/data/en/" + Id[-1] + "/" + temp, 'wb') as s:
                    s.write(data)

                en_file.append(temp)
            else:
                en_file.append("null")
        elif ptr == 8:
            more_information.append(data_td[ptr].getText())
        elif ptr == 9:
            temp = data_td[ptr].getText()
            
            video_address.append(temp)
        elif ptr == 10:
            if(data_td[ptr].getText() == ""):
                attention.append("無")
            else:
                attention.append(data_td[ptr].getText())

100%|███████████████████████████████████████████| 99/99 [04:18<00:00,  2.61s/it]


In [9]:
df = pd.DataFrame({"Co_ID" : Id, "Co_name" : name, "Date" : date, "Time" : time, "Form" : place, "Message" : message
                          , "CH_file" : ch_file, "EN_file" : en_file, "More information" : more_information, "Video address" : video_address, "Attention" : attention})

In [10]:
df.head(10)

,Co_ID,Co_name,Date,Time,Form,Message,CH_file,EN_file,More information,Video address,Attention
0,2801,彰銀,2022-06-29,14:30,本次為中文線上法說會，連結網址詳如本公司投資人關係專區或如後說明。,報告本行2022年第1季財務資訊及經營成果,null,null,https://www.bankchb.com/frontend/earningsCall.jsp,\n同步收看網址\n：\nhttps://www.zucast.com/event/hwDb...,無
1,4571,鈞興-KY,2022-06-22,14:30,線上法人說明會,本公司受邀參加中國信託證券舉辦之線上法人說明會，說明公司營運回顧及業務展望,null,null,http://www.khgears.com/ft/gudong.php?id=25,,無
2,2884,玉山金,2022-06-21,16:00,線上,2022年第1季財業務資訊,null,null,https://www.esunfhc.com/zh-tw/investor-relatio...,,無
3,1590,亞德客-KY,2022-06-21,09:00,線上法說會,公司簡介與營運狀況說明,159020220510M002.pdf,159020220510E002.pdf,https://as-tw.airtac.com/about.aspx?c_kind=5&c...,,受邀參加瑞銀證券舉辦之線上法說會「Taiwan Virtual Conference 2022」。
4,2449,京元電子,2022-06-21,09:00,線上法說會,本公司111年06月21日受邀參加瑞銀證券舉辦的UBS Taiwan Virtual Con...,244920220516M001.pdf,244920220516E001.pdf,無,,無
5,2330,台積電,2022-06-20,16:00,線上機構投資人說明會,本公司受邀參加瑞銀證券所舉辦之『UBS Taiwan Virtual Conference ...,233020220414M001.pdf,233020220414E001.pdf,https://investor.tsmc.com/chinese/quarterly-re...,,無。
6,2360,致茂,2022-06-20,14:00,電話會議,本公司受邀參加瑞銀證券舉辦之UBS Taiwan Virtual Conference 20...,236020220428M001.pdf,236020220428E001.pdf,http://www.chromaate.com,,無
7,2891,中信金,2022-06-20,10:00,線上,本公司將於111年06月20日參加UBS舉辦之投資人說明會(Taiwan Virtual C...,289120220518M001.pdf,289120220518E001.pdf,http://ir.ctbcholding.com/c/events.php,,無
8,4961,天鈺,2022-06-20,10:00,線上機構投資人說明會,本公司111/6/20受邀參加瑞銀證券舉辦之「Taiwan Virtual Conferen...,496120220506M001.pdf,496120220506E001.pdf,https://www.fitipower.com/investment.asp?id=12,,無
9,1102,亞泥,2022-06-20,09:00,線上法人說明會,本公司受邀參加UBS舉辦Taiwan Virtual Conference 2022,110220220601M001.pdf,110220220601E001.pdf,https://www.acc.com.tw/,,無


In [11]:
df.to_csv("/home/cosbi/桌面/financialData/twseData/dataFrame/2022-06.csv", index = False)